In [2]:
# 데이터프레임을 다루기 위한 라이브러리를 불러옵니다.
import numpy as np
import pandas as pd
import re

# 시각화를 위한 라이브러리를 불러옵니다.
import seaborn as sns
import math
import matplotlib.pyplot as plt

# 정규성, 등분산성과 같은 데이터 분포를 살펴보기 위한 라이브러리를 불러옵니다.
from scipy import stats
from scipy.stats import shapiro
from scipy.stats import anderson
from scipy.stats import bartlett
from scipy.stats import levene
from scipy.stats import pearsonr
import statsmodels.api as sm
from statsmodels.formula.api import ols
import 

# 원하는 csv파일을 불러옵니다.
df = pd.read_csv('데이터 최종본(완) (1).csv')
pd.set_option('display.max_rows', None)

# 새로운 열 이름을 생성하고 열 이름을 변경합니다.
new_columns = ['강좌명','수강생수','가격','평점','수강평수','난이도','강의시간','강의게시일']
df.columns = new_columns


# 각각의 속성을 원하는 시각화를 위해 형태를 바꿔줍니다. -> 시각화를 위한 데이터프레임을 생성합니다.
df['강좌명'] = df['강좌명'].str.replace('\n대시보드', '')
df['강의게시일'] = pd.to_datetime(df['강의게시일'].str.extract(r'(\d+)년 (\d+)월 (\d+)일').agg('-'.join, axis=1))
df['수강평수'] = df['수강평수'].str.extract(r'(\d+)개의').fillna(0).astype(int)
df['수강생수'] = df['수강생수'].str.replace('명', '')
df['수강생수'] = df['수강생수'].str.replace(',', '').astype(int)
df['가격'] = df['가격'].replace('무료', '₩0')
df['가격'] = df['가격'].str.extract(r'₩([^₩]+)$')
df['가격'] = df['가격'].str.replace(',', '').astype(int)
df['시간'] = df['강의시간'].str.extract(r'(\d+)시간', expand=False).fillna(0)
df['분'] = df['강의시간'].str.extract(r'(\d+)분', expand=False).fillna(0)
df['강의시간_분'] = df['시간'].astype(int)*60 + df['분'].astype(int)
df = df.drop(['시간', '분', '강의시간'], axis=1)
df['평점'] = df['평점'].str.extract(r'([\d.]+)').fillna(0).astype(float)
df['무료/유료'] = df['가격'].apply(lambda x: '무료' if x == 0 else '유료')
df['평가지수'] = df['수강평수'] * df['평점']
df = df[df['강의시간_분'] != 0]
df['강의시간당가격'] = df.apply(lambda row: int(row['가격'] / (row['강의시간_분'] / 60)), axis=1)
df = df[df['평가지수'] != 0]
# 중복되는 행을 제거해줍니다.
df = df.drop_duplicates('강좌명', keep='first')
#df.to_csv('데이터 시각화용.csv', index=False)


df_regression = df.copy()
df_regression = pd.concat([df_regression, pd.get_dummies(df_regression['난이도'], prefix='난이도')], axis=1)
df_regression['유무료'] = df['무료/유료'].apply(lambda x: 1 if x == '유료' else 0) #유료(1)/무료(0)
df_regression= df_regression.drop(['난이도', '무료/유료'], axis=1)

df_regression['로그_수강생수'] = np.log1p(df_regression['수강생수'])
df_regression['로그_가격'] = np.log1p(df_regression['가격'])
df_regression['로그_강의시간'] = np.log1p(df_regression['강의시간_분'])
df_regression

,강좌명,수강생수,가격,평점,수강평수,강의게시일,강의시간_분,평가지수,강의시간당가격,난이도_입문자,난이도_중급자,난이도_초급자,유무료,로그_수강생수,로그_가격,로그_강의시간
0,스프링 핵심 원리 - 기본편,30486,61600,5.0,326,2020-09-21,725,1630.0,5097,False,False,True,1,10.325056,11.028433,6.587550
1,김영한의 실전 자바 - 기본편,5147,30800,5.0,205,2023-11-28,1011,1025.0,1827,False,False,True,1,8.546364,10.335302,6.919684
2,스프링 부트 - 핵심 원리와 활용,8595,69300,5.0,203,2023-02-28,945,1015.0,4400,False,False,True,1,9.059052,11.146215,6.852243
3,모든 개발자를 위한 HTTP 웹 기본 지식,27573,30800,5.0,832,2020-12-20,340,4160.0,5435,False,False,True,1,10.224629,10.335302,5.831882
4,[2024 NEW] 개발자를 위한 쉬운 도커,170,61600,5.0,3,2024-01-08,594,15.0,6222,False,False,True,1,5.141664,11.028433,6.388561
5,스프링 MVC 1편 - 백엔드 웹 개발 핵심 기술,22558,69300,5.0,724,2021-03-07,922,3620.0,4509,False,False,True,1,10.023889,11.146215,6.827629
6,홍정모의 따라하며 배우는 C언어,4917,99000,4.9,342,2019-12-02,3065,1675.8,1938,True,False,False,1,8.500657,11.502885,8.028129
7,Next + React Query로 SNS 서비스 만들기,1330,69300,4.6,14,2023-12-04,886,64.4,4693,False,True,False,1,7.193686,11.146215,6.787845
8,프로그래밍 시작하기 : 파이썬 입문 (Inflearn Original),24940,33000,4.7,823,2019-06-02,857,3868.1,2310,True,False,False,1,10.124268,10.404293,6.754604
9,10주완성 C++ 코딩테스트 | 알고리즘 코딩테스트,2867,165000,4.8,174,2021-12-23,1550,835.2,6387,False,False,True,1,7.961370,12.013707,7.346655


In [5]:
from scipy.stats import mannwhitneyu
import plotly.express as px

In [7]:
df2 = df_regression.sort_values(by='로그_가격')
# 강의시간별 누적 강의인원 계산
df2['누적 강의인원'] = df2['수강생수'].cumsum()
# Plotly를 사용하여 그래프 그리기
fig = px.line(df2, x='로그_가격', y='누적 강의인원', title='가격별 누적 강의인원')
fig.show()

In [17]:
group_1 = df_regression[df_regression['로그_가격'] >= 10]
group_2 = df_regression[df_regression['로그_가격'] < 10 ]
# Mann-Whitney U 검정 수행
mannwhitneyu(group_1['수강생수'], group_2['수강생수'])

MannwhitneyuResult(statistic=219229.5, pvalue=1.222702860547179e-37)

In [9]:
df3= df_regression.sort_values(by='로그_강의시간')
# 강의시간별 누적 강의인원 계산
df3['누적 강의인원'] = df3['수강생수'].cumsum()
# Plotly를 사용하여 그래프 그리기
fig = px.line(df3, x='로그_강의시간', y='누적 강의인원', title='강의시간별 누적 강의인원')
fig.show()

In [10]:
group_1 = df_regression[df_regression['로그_강의시간'] >= 7.3]
group_2 = df_regression[df_regression['로그_강의시간'] < 7.3]
# Mann-Whitney U 검정 수행
mannwhitneyu(group_1['수강생수'], group_2['수강생수'])

MannwhitneyuResult(statistic=72970.5, pvalue=0.0021485797477855733)

In [14]:
group_1 = df_regression[df_regression['로그_강의시간'] >= 5.52]
group_2 = df_regression[df_regression['로그_강의시간'] < 5.52]
# Mann-Whitney U 검정 수행
mannwhitneyu(group_1['수강생수'], group_2['수강생수'])

MannwhitneyuResult(statistic=380959.0, pvalue=1.4422922488844678e-05)

In [11]:
df4 = df_regression.sort_values(by='평점')
# 강의시간별 누적 강의인원 계산
df4['누적 강의인원'] = df4['수강생수'].cumsum()
# Plotly를 사용하여 그래프 그리기
fig = px.line(df4, x='평점', y='누적 강의인원', title='강의시간별 누적 강의인원')
fig.show()

In [16]:
sample1 = df_regression['평점'] < 4
sample2 = df_regression['평점'] >= 4
print(mannwhitneyu(sample1,sample2))

MannwhitneyuResult(statistic=112398.0, pvalue=0.0)
